In [1]:
!pip install llmlingua openai

In [2]:
import pandas as pd

In [3]:
import getpass
import openai

secret_key = getpass.getpass('Digite sua chave da OpenAI: ')
openai.api_key = secret_key

Digite sua chave da OpenAI: ··········


In [4]:
client = openai.Client(api_key=secret_key)

In [5]:
def analyze_sentiment_openai(text):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Analyze the sentiment of the following text: '{text}'"}
        ],
        max_tokens=60
    )
    sentiment = response.choices[0].message.content.strip()
    return sentiment

In [7]:
from llmlingua import PromptCompressor

In [8]:
llm_lingua = PromptCompressor()

def analyze_sentiment_llmlingua(text):
    compressed_prompt = llm_lingua.compress_prompt(
        f"Analyze the sentiment of the following text: '{text}'",
        instruction="",
        question="",
        target_token=100  # Reduzindo o número de tokens alvo
    )
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": compressed_prompt['compressed_prompt']}
        ],
        max_tokens=60
    )
    sentiment = response.choices[0].message.content.strip()
    return sentiment

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

In [9]:
# Exemplo de textos para análise
texts = ["I love this product!", "This is the worst experience ever.", "I'm feeling neutral about this."]

# Analisando com OpenAI
results_openai = [analyze_sentiment_openai(text) for text in texts]

# Analisando com LLMLingua
results_llmlingua = [analyze_sentiment_llmlingua(text) for text in texts]

# Criando um DataFrame para comparação
df_results = pd.DataFrame({
    'Text': texts,
    'Sentiment_OpenAI': results_openai,
    'Sentiment_LLMLingua': results_llmlingua
})

print(df_results)

                                 Text  \
0                I love this product!   
1  This is the worst experience ever.   
2     I'm feeling neutral about this.   

                                    Sentiment_OpenAI  \
0  The sentiment of the text "I love this product...   
1  The sentiment of the text "This is the worst e...   
2  The sentiment of the text "I'm feeling neutral...   

                                 Sentiment_LLMLingua  
0  The sentiment of the text "I love this product...  
1  The sentiment of the text 'This is the worst e...  
2  The sentiment of the text "I'm feeling neutral...  
